# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# pip install --user python-dotenv


In [1]:
%load_ext dotenv
%dotenv 


In [3]:
# pip install --user dask


In [5]:
# pip install --user pandas


In [2]:
import pandas as pd
import dask

In [3]:
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.


PRICE_DATA = os.getenv("PRICE_DATA")
import shutil
if os.path.exists(PRICE_DATA):
    shutil.rmtree(PRICE_DATA)
PRICE_DATA



'../../05_src/data/prices/'

In [14]:
# for ticker in px_dt.ticker.unique():
#     ticker_dt = px_dt[px_dt.ticker == ticker]
#     ticker_dt = ticker_dt.assign(year = ticker_dt.Date.dt.year)
#     for yr in ticker_dt.year.unique():
#         yr_dt = ticker_dt[ticker_dt.year == yr]
#         yr_path = os.path.join(PRICE_DATA, ticker, f"{ticker}_{yr}.parquet")
#         os.makedirs(os.path.dirname(yr_path), exist_ok=True)
#         yr_dt.to_parquet(yr_path, engine = "pyarrow")
    
from glob import glob

parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
parquet_files[0]

# dd_px = dd.read_parquet(parquet_files).set_index("ticker")

'../../05_src/data/prices/AMZN/AMZN_2023.parquet'

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [19]:
# Write your code below.
p_temp= '../../05_src/data/prices/EXPE/EXPE_2024.parquet'
df=pd.read_parquet(p_temp)
df.head()

# df.columns




,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year
index,,,,,,,,,,
2537,2024-01-02 00:00:00-05:00,151.669998,152.380005,147.240005,148.759995,2897900,0.0,0.0,EXPE,2024
2538,2024-01-03 00:00:00-05:00,146.610001,148.470001,144.899994,144.990005,4108200,0.0,0.0,EXPE,2024
2539,2024-01-04 00:00:00-05:00,145.990005,147.339996,144.559998,144.820007,2709600,0.0,0.0,EXPE,2024
2540,2024-01-05 00:00:00-05:00,144.279999,147.550003,144.279999,146.320007,1579600,0.0,0.0,EXPE,2024
2541,2024-01-08 00:00:00-05:00,147.110001,150.470001,146.029999,149.679993,2333900,0.0,0.0,EXPE,2024


In [22]:
import numpy as np

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range= lambda x: x['High']-x['Low']
))

/var/folders/_y/d081_vb93vn25b4kmt3n35c00000gn/T/ipykernel_7718/2020700850.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [24]:
df_feat=dd_feat.compute()

# Add a rolling average return calculation with a window of 10 days
df_feat['rolling_avg_return'] = df_feat['returns'].rolling(10).mean()


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

No, it was not necessary to convert to pandas to calculate the moving average return. It's better to perform such operations directly in Dask, especially when working with large datasets that may not fit into memory. Dask is designed to handle large-scale data processing in parallel, which can provide significant performance benefits over pandas for large datasets.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.